In [9]:
# Ingest data

In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def pm25_template_table(date):
    row_list = []
    columns = ['เขต', 'สถานีตรวจวัด', 'วันที่และเวลา', 'PM10', 'PM2.5', 'CO', 'NO2',
                'O3', 'Wind Speed', 'Wind Direction', 'Temperature',
                'Relative Humidity', 'Air Pressure']
    for i in range(24):
        datetime_str = f"{date} {str(i).zfill(2)}:00"
        row = [None,None,datetime_str] + [None for i in range(len(columns)-3)]
        row_list.append(row)
    return pd.DataFrame(row_list, columns=columns)


#### Create background data (for replacing null values)

In [3]:
# Get all CSV files recursively
csv_files = glob.glob(f"../data/**/**/*.csv", recursive=True)

# Ensure there are CSV files before proceeding
if not csv_files:
    print("No CSV files found. Exiting.")
    exit()

# Concatenate all csv files in the specified folder
df_list = []
for file in csv_files:
    is_empty = False
    try:
        temp_df = pd.read_csv(file)
        is_empty = True if temp_df.empty else False
    except:
        is_empty = True

    if is_empty:
        date = file.split('/')[-1][:10]
        year,month,day = date.split('-')
        date_re = f'{int(day)}/{int(month)}/{year}'
        temp_df = pm25_template_table(date_re)

    df_list.append(temp_df)

concat_df = pd.concat(df_list, axis=0) # Warning occurs since df contains all-NA columns


/tmp/ipykernel_37932/2653709166.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  concat_df = pd.concat(df_list, axis=0) # Warning occurs since df contains all-NA columns


In [4]:
concat_df['datetime'] = pd.to_datetime(concat_df['วันที่และเวลา'], format="%d/%m/%Y %H:%M")
concat_df['month'] = concat_df['datetime'].dt.month
concat_df['day'] = concat_df['datetime'].dt.day
concat_df['hour'] = concat_df['datetime'].dt.hour

background_df = concat_df.groupby(['month', 'day', 'hour']).mean(numeric_only=True).reset_index()
background_df['datetime'] = pd.to_datetime(background_df[['month', 'day', 'hour']].assign(year=2024))

In [5]:
background_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   month              8784 non-null   int32         
 1   day                8784 non-null   int32         
 2   hour               8784 non-null   int32         
 3   PM10               4505 non-null   float64       
 4   PM2.5              8775 non-null   float64       
 5   NO2                8775 non-null   float64       
 6   O3                 8775 non-null   float64       
 7   Wind Speed         8775 non-null   float64       
 8   Wind Direction     8775 non-null   float64       
 9   Temperature        8775 non-null   float64       
 10  Relative Humidity  8775 non-null   float64       
 11  Air Pressure       8775 non-null   float64       
 12  datetime           8784 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(9), int32(3)
memory usage: 789.3 

In [6]:
background_df

,month,day,hour,PM10,PM2.5,NO2,O3,Wind Speed,Wind Direction,Temperature,Relative Humidity,Air Pressure,datetime
0,1,1,0,61.2,43.266667,39.0,17.0,0.412500,172.625000,28.500000,63.625000,1012.750000,2024-01-01 00:00:00
1,1,1,1,63.4,55.144444,35.0,19.0,0.487500,169.375000,28.275000,63.625000,1012.750000,2024-01-01 01:00:00
2,1,1,2,87.6,45.522222,35.0,19.0,0.450000,139.375000,28.237500,62.375000,1012.125000,2024-01-01 02:00:00
3,1,1,3,64.8,37.711111,36.0,16.0,0.425000,117.375000,27.900000,63.250000,1011.875000,2024-01-01 03:00:00
4,1,1,4,55.8,36.900000,30.0,16.0,0.275000,107.625000,27.200000,67.750000,1011.125000,2024-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,12,31,19,38.0,25.500000,36.0,47.0,0.466667,124.888889,28.844444,45.444444,1011.666667,2024-12-31 19:00:00
8780,12,31,20,44.8,32.220000,35.5,45.5,0.355556,236.222222,27.955556,49.333333,1012.000000,2024-12-31 20:00:00
8781,12,31,21,52.0,35.710000,48.5,14.0,0.211111,208.555556,27.233333,52.111111,1012.888889,2024-12-31 21:00:00
8782,12,31,22,64.2,40.830000,39.5,13.5,0.233333,208.666667,26.588889,55.666667,1013.333333,2024-12-31 22:00:00


In [7]:
# fill null values using forward fill
background_df = background_df.sort_values(by='datetime')
background_df = background_df.ffill()


In [8]:
background_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   month              8784 non-null   int32         
 1   day                8784 non-null   int32         
 2   hour               8784 non-null   int32         
 3   PM10               8784 non-null   float64       
 4   PM2.5              8784 non-null   float64       
 5   NO2                8784 non-null   float64       
 6   O3                 8784 non-null   float64       
 7   Wind Speed         8784 non-null   float64       
 8   Wind Direction     8784 non-null   float64       
 9   Temperature        8784 non-null   float64       
 10  Relative Humidity  8784 non-null   float64       
 11  Air Pressure       8784 non-null   float64       
 12  datetime           8784 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(9), int32(3)
memory usage: 789.3 

In [11]:
background_df.to_csv(r'../background_data/background_data_20250216.csv', index=False)